In [2]:
import subprocess
from langchain_openai import OpenAI
from dotenv import load_dotenv
import os 
from typing import List
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


In [3]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [4]:
class Code(BaseModel):
    Filename: str = Field(description="what is the Main function name")
    code: str = Field(description="answer to resolve the problem statement")

In [22]:
class AIAgent:
    def __init__(self):
        self.model = ChatOpenAI(model="gpt-4")
        self.parser = PydanticOutputParser(pydantic_object=Code)
        self.template=PromptTemplate(
            template="You are a helpful AI coding Assistant,You need to generate code for the following query,\n {format_instructions}\n {query}",
            input_variables=["query"],
            partial_variables={"format_instructions": self.parser.get_format_instructions()},
        )
        self.chain = self.template|self.model|self.parser
    def run_code(self,lang,filename,inputs=None):
        
        cmdline=subprocess.run([lang,filename],text=True,capture_output=True,input=inputs)
        if cmdline.returncode != 0:
            return cmdline.stderr
        else:
            return cmdline.stdout
    def gen_code(self,query,):
        answer =self.chain.invoke(query)
        
        with open(answer.Filename, "w") as file:
            file.write(answer.code)    
        return answer.Filename                         
    def rectify(self):
        None

In [23]:
agent=AIAgent()
filename=agent.gen_code("generate a java code with a function to calculate 3x3 matrix multiplications.")

In [24]:
agent.run_code(lang="java",filename=filename)

'6 6 6 \n12 12 12 \n18 18 18 \n'